In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from sklearn.decomposition import PCA

读入原始的数据加上label后的数据

In [16]:
df = pd.read_excel('./dataset20220613_label.xlsx')
print(df.shape)
df.sample(5)

(1528, 90)


,Filnum,IDps,condition,Y,ltV,leV,trV,leS,trS,leT,...,"trP(1,:)",Unnamed: 81,Unnamed: 82,"trP(2,:)",Unnamed: 84,Unnamed: 85,"trP(3,:)",Unnamed: 87,Unnamed: 88,label
639,40,0.008095,5,2,-3.049242,11.184971,9.609992,1152.0520,1249.299,103,...,-0.001238,-0.000375,-0.003266,0.000249,-0.001137,0.005302,0.000202,0.000770,0.000244,2
19,1,0.915122,1,1,-3.315667,10.019797,10.681292,1182.3360,1206.986,118,...,-0.000393,-0.002348,-0.001367,0.000831,0.006981,-0.010643,0.000220,0.000444,0.000185,0
689,43,0.719793,6,1,-4.410399,12.828090,11.681455,1282.8090,1308.323,100,...,0.000062,-0.007409,-0.000402,0.000887,0.001687,-0.010665,0.000256,0.001190,0.000008,0
53,6,0.986510,1,1,-3.753544,11.828340,11.393243,1253.8040,1219.077,106,...,0.000080,-0.000656,0.002304,0.001104,0.004260,-0.012615,0.000030,-0.000297,0.001202,0
460,25,0.951342,4,1,-3.771205,11.425886,11.308739,1256.8475,1255.270,110,...,-0.001809,-0.005398,-0.003224,0.002926,0.014665,-0.012431,0.000070,-0.000018,0.001084,0


In [17]:
df.columns

Index(['Filnum', 'IDps', 'condition', 'Y', 'ltV', 'leV', 'trV', 'leS', 'trS',
       'leT', 'trT', 'leToeUi', 'leswpht', 'trToeUi', 'trswpht', 'leToeMa',
       'trToeMa', 'pelv(1,:)', 'Unnamed: 18', 'Unnamed: 19', 'pelv(2,:)',
       'Unnamed: 21', 'Unnamed: 22', 'pelv(3,:)', 'Unnamed: 24', 'Unnamed: 25',
       'pelv(4,:)', 'Unnamed: 27', 'Unnamed: 28', 'pelv(5,:)', 'Unnamed: 30',
       'Unnamed: 31', 'pelv(6,:)', 'Unnamed: 33', 'Unnamed: 34', 'leA(1,:)',
       'Unnamed: 36', 'Unnamed: 37', 'leA(2,:)', 'Unnamed: 39', 'Unnamed: 40',
       'leA(3,:)', 'Unnamed: 42', 'Unnamed: 43', 'trA(1,:)', 'Unnamed: 45',
       'Unnamed: 46', 'trA(2,:)', 'Unnamed: 48', 'Unnamed: 49', 'trA(3,:)',
       'Unnamed: 51', 'Unnamed: 52', 'leM(1,:)', 'Unnamed: 54', 'Unnamed: 55',
       'leM(2,:)', 'Unnamed: 57', 'Unnamed: 58', 'leM(3,:)', 'Unnamed: 60',
       'Unnamed: 61', 'trM(1,:)', 'Unnamed: 63', 'Unnamed: 64', 'trM(2,:)',
       'Unnamed: 66', 'Unnamed: 67', 'trM(3,:)', 'Unnamed: 69', 'Unnamed: 7

## 特征工程

逐一分析

通用的特征工程步骤有：

1. Imputation： 缺失值过多需要插值，本例缺失值不多，不需要插值

2. Handling Outliers： 有两种检测异常值的方法，一是值在均值+3σ范围外的认为是异常值，二是值在0.05分位数和0.95分位数的认为是异常值。本例计划对数值类变量进行异常值检测和筛除。

3. Binning 可以把连续值转换成类别变量，比如0-30的值分为类别0，30-60的值分为类别1。这样坐的目的是增加模型的鲁棒性，减少过拟合。本例我选择不使用。

4. Log Transform 把变量x变成log(x)，这样做的目的是为了让变量分布更正态，使得模型更加鲁棒。本例我选择不使用。

5. One-Hot Encoding 将类别变量转换成one-hot编码，本例对condition变量使用。

6. Grouping Operations 合并不同的数据表。本例只有一个数据表，不需要合并。

7. Feature Split 把一个变量分成两个变量。如果有的变量表示了多种信息需要这么做，本例不需要。

8. Scaling 将变量放缩到同一个尺度下。有两种方式，normalization和standardization。normalization是min-max normalization，standardization是z-score normalization。本例选择normalization。

9. Extracting Date 本例没有日期变量，不需要提取日期。

10. Dimensionality Reduction 特征太多难以分析，增加训练时间。可以通过降维方法PCA或者特征选择选择最重要的特征降维等。本例听取建议不降维。

    1. 观察类别变量’conditon和Y是否每个类别的样本数均衡。

In [18]:
df['Y'].value_counts()

1    889
2    639
Name: Y, dtype: int64

In [19]:
df['condition'].value_counts()

3     138
1     136
6     135
4     131
8     128
2     127
10    127
9     126
12    126
5     122
11    119
7     113
Name: condition, dtype: int64

观察结果没有样本不均衡现象

    2. 对condition变量进行onehot编码

In [20]:
encoder = OneHotEncoder()
onehotarray = encoder.fit_transform(df[['condition']]).toarray()
items = [f'condition_{item}' for item in encoder.categories_[0]]
df[items] = onehotarray
df.sample(5)

,Filnum,IDps,condition,Y,ltV,leV,trV,leS,trS,leT,...,condition_3,condition_4,condition_5,condition_6,condition_7,condition_8,condition_9,condition_10,condition_11,condition_12
1425,92,0.190995,12,1,-3.285937,10.224420,9.312496,1216.7060,1238.562,119,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
852,51,0.150685,7,1,-3.183281,10.438743,9.725672,1179.5780,1244.886,113,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
692,46,0.425027,6,1,-4.517601,13.086672,11.780869,1269.4072,1260.553,97,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1249,80,0.153521,10,2,-3.217858,10.163741,8.682182,1311.1226,1284.963,129,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1073,66,0.191069,9,1,-3.332514,10.200570,10.147854,1162.8650,1319.221,114,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


可以看到df最右侧的列是condition的onehot编码，共12列

    3. 类别变量Y的二分类用0和1表示

In [21]:
df['Y'] = df['Y'] - 1

    4. 丢弃掉数据里多余的数据列，'Filnum', 'IDps', 'condition'

In [22]:
df.drop(['Filnum'], axis=1, inplace=True)
df.drop(['IDps'], axis=1, inplace=True)
df.drop(['condition'], axis=1, inplace=True)
df.sample(5)

,Y,ltV,leV,trV,leS,trS,leT,trT,leToeUi,leswpht,...,condition_3,condition_4,condition_5,condition_6,condition_7,condition_8,condition_9,condition_10,condition_11,condition_12
831,0,-3.335831,9.915836,9.969393,1209.732,1345.868,122,135,4,118,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
710,0,-3.957022,11.661816,10.057481,1329.447,1297.415,114,129,6,108,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1400,1,-3.161513,10.587761,9.475647,1238.768,1260.261,117,133,118,-1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
449,0,-3.955776,12.282089,11.858183,1240.491,1233.251,101,104,68,33,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579,0,-2.849398,9.922290,8.904993,1230.364,1237.794,124,139,9,115,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


    5. 异常值检测和去除异常值

采用均值方差法去除异常值

In [23]:
number_cols = ['ltV', 'leV', 'trV', 'leS', 'trS',
       'leT', 'trT', 'leToeUi', 'leswpht', 'trToeUi', 'trswpht', 'leToeMa',
       'trToeMa', 'pelv(1,:)', 'Unnamed: 18', 'Unnamed: 19', 'pelv(2,:)',
       'Unnamed: 21', 'Unnamed: 22', 'pelv(3,:)', 'Unnamed: 24', 'Unnamed: 25',
       'pelv(4,:)', 'Unnamed: 27', 'Unnamed: 28', 'pelv(5,:)', 'Unnamed: 30',
       'Unnamed: 31', 'pelv(6,:)', 'Unnamed: 33', 'Unnamed: 34', 'leA(1,:)',
       'Unnamed: 36', 'Unnamed: 37', 'leA(2,:)', 'Unnamed: 39', 'Unnamed: 40',
       'leA(3,:)', 'Unnamed: 42', 'Unnamed: 43', 'trA(1,:)', 'Unnamed: 45',
       'Unnamed: 46', 'trA(2,:)', 'Unnamed: 48', 'Unnamed: 49', 'trA(3,:)',
       'Unnamed: 51', 'Unnamed: 52', 'leM(1,:)', 'Unnamed: 54', 'Unnamed: 55',
       'leM(2,:)', 'Unnamed: 57', 'Unnamed: 58', 'leM(3,:)', 'Unnamed: 60',
       'Unnamed: 61', 'trM(1,:)', 'Unnamed: 63', 'Unnamed: 64', 'trM(2,:)',
       'Unnamed: 66', 'Unnamed: 67', 'trM(3,:)', 'Unnamed: 69', 'Unnamed: 70',
       'leP(1,:)', 'Unnamed: 72', 'Unnamed: 73', 'leP(2,:)', 'Unnamed: 75',
       'Unnamed: 76', 'leP(3,:)', 'Unnamed: 78', 'Unnamed: 79', 'trP(1,:)',
       'Unnamed: 81', 'Unnamed: 82', 'trP(2,:)', 'Unnamed: 84', 'Unnamed: 85',
       'trP(3,:)', 'Unnamed: 87', 'Unnamed: 88']
factor = 4
data = df.copy()
for col in number_cols:
    upper_lim = data[col].mean () + data[col].std () * factor
    lower_lim = data[col].mean () - data[col].std () * factor
    data = data[(data[col] < upper_lim) & (data[col] > lower_lim)]
data.shape

(1346, 99)

    6. 尺度放缩,只针对数值变量

In [24]:
from sklearn.preprocessing import MinMaxScaler
number_data = MinMaxScaler().fit_transform(data[number_cols])
data[number_cols] = number_data
data.describe()

,Y,ltV,leV,trV,leS,trS,leT,trT,leToeUi,leswpht,...,condition_3,condition_4,condition_5,condition_6,condition_7,condition_8,condition_9,condition_10,condition_11,condition_12
count,1346.00000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,...,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000,1346.000000
mean,0.43685,0.603527,0.418131,0.459886,0.487405,0.487510,0.420208,0.417676,0.436734,0.365475,...,0.088410,0.080238,0.080981,0.088410,0.078009,0.086181,0.082467,0.080981,0.078009,0.087667
std,0.49618,0.148877,0.164496,0.191000,0.159823,0.156684,0.181845,0.200609,0.235056,0.260114,...,0.283996,0.271762,0.272907,0.283996,0.268285,0.280736,0.275176,0.272907,0.268285,0.282915
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,0.497742,0.309104,0.314651,0.388767,0.384229,0.290909,0.232877,0.439716,0.255319,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,0.604966,0.404334,0.434909,0.478422,0.468646,0.418182,0.445205,0.468085,0.326241,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.00000,0.716120,0.523487,0.614950,0.567247,0.592919,0.527273,0.561644,0.503546,0.397163,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


    7. 写一个函数转换一下名字，将类似pelv(1,:)后的Unnamed转换为pelv(1,:)_1, pelv(1,:)_2

In [25]:
cols = data.columns
pre = ''
i = 0
while i < len(cols):
    if cols[i].find('Unnamed')>=0:
        data.rename(columns={cols[i]:pre+'_1'}, inplace=True)
        data.rename(columns={cols[i+1]:pre+'_2'}, inplace=True)
        i+=2
    else:
        pre = cols[i]
        i+=1
        
data.columns

Index(['Y', 'ltV', 'leV', 'trV', 'leS', 'trS', 'leT', 'trT', 'leToeUi',
       'leswpht', 'trToeUi', 'trswpht', 'leToeMa', 'trToeMa', 'pelv(1,:)',
       'pelv(1,:)_1', 'pelv(1,:)_2', 'pelv(2,:)', 'pelv(2,:)_1', 'pelv(2,:)_2',
       'pelv(3,:)', 'pelv(3,:)_1', 'pelv(3,:)_2', 'pelv(4,:)', 'pelv(4,:)_1',
       'pelv(4,:)_2', 'pelv(5,:)', 'pelv(5,:)_1', 'pelv(5,:)_2', 'pelv(6,:)',
       'pelv(6,:)_1', 'pelv(6,:)_2', 'leA(1,:)', 'leA(1,:)_1', 'leA(1,:)_2',
       'leA(2,:)', 'leA(2,:)_1', 'leA(2,:)_2', 'leA(3,:)', 'leA(3,:)_1',
       'leA(3,:)_2', 'trA(1,:)', 'trA(1,:)_1', 'trA(1,:)_2', 'trA(2,:)',
       'trA(2,:)_1', 'trA(2,:)_2', 'trA(3,:)', 'trA(3,:)_1', 'trA(3,:)_2',
       'leM(1,:)', 'leM(1,:)_1', 'leM(1,:)_2', 'leM(2,:)', 'leM(2,:)_1',
       'leM(2,:)_2', 'leM(3,:)', 'leM(3,:)_1', 'leM(3,:)_2', 'trM(1,:)',
       'trM(1,:)_1', 'trM(1,:)_2', 'trM(2,:)', 'trM(2,:)_1', 'trM(2,:)_2',
       'trM(3,:)', 'trM(3,:)_1', 'trM(3,:)_2', 'leP(1,:)', 'leP(1,:)_1',
       'leP(1,:)_2', 'leP

    7. 保存数据

In [26]:
data.to_csv('./data_final.csv', index=False)